# Day 19: Beacon Scanner

As your probe drifted down through this area, it released an assortment of beacons and scanners into the water. It's difficult to navigate in the pitch black open waters of the ocean trench, but if you can build a map of the trench using data from the scanners, you should be able to safely reach the bottom.

The beacons and scanners float motionless in the water; they're designed to maintain the same position for long periods of time. Each scanner is capable of detecting all beacons in a large cube centered on the scanner; beacons that are at most 1000 units away from the scanner in each of the three axes (x, y, and z) have their precise position determined relative to the scanner. However, scanners cannot detect other scanners. The submarine has automatically summarized the relative positions of beacons detected by each scanner (your puzzle input).

For example, if a scanner is at x,y,z coordinates 500,0,-500 and there are beacons at -500,1000,-1500 and 1501,0,-500, the scanner could report that the first beacon is at -1000,1000,-1000 (relative to the scanner) but would not detect the second beacon at all.

Unfortunately, while each scanner can report the positions of all detected beacons relative to itself, the scanners do not know their own position. You'll need to determine the positions of the beacons and scanners yourself.

The scanners and beacons map a single contiguous 3d region. This region can be reconstructed by finding pairs of scanners that have overlapping detection regions such that there are at least 12 beacons that both scanners detect within the overlap. By establishing 12 common beacons, you can precisely determine where the scanners are relative to each other, allowing you to reconstruct the beacon map one scanner at a time.

Unfortunately, there's a second problem: the scanners also don't know their rotation or facing direction. Due to magnetic alignment, each scanner is rotated some integer number of 90-degree turns around all of the x, y, and z axes. That is, one scanner might call a direction positive x, while another scanner might call that direction negative y. Or, two scanners might agree on which direction is positive x, but one scanner might be upside-down from the perspective of the other scanner. In total, each scanner could be in any of 24 different orientations: facing positive or negative x, y, or z, and considering any of four directions "up" from that facing.

## Data Structures

In [ ]:
record class Pos(long x, long y, long z) {
    Lazy<decimal> _mag = new(() => (decimal)Math.Sqrt(x*x + y*y + z*z));
    public decimal Mag() => _mag.Value;
    public override string ToString()
        => $"{x}/{y}/{z}";
    public Pos Add(Pos with)
        => with == null ? this : new Pos(x + with.x, y + with.y, z + with.z);
    public Pos Diff(Pos with)
        => with == null ? this : new Pos(x - with.x, y - with.y, z - with.z);
    public bool Match(Pos with)
        => with != null && x == with.x && y == with.y && z == with.z;
    public Pos Reorient(int[] o)
        => new Pos(
            x * o[0] + y * o[1] + z * o[2],
            x * o[3] + y * o[4] + z * o[5],
            x * o[6] + y * o[7] + z * o[8]);
    static int S(long a, long b)
        => a == b ? 1 : a == -b ? -1 : 0;
    public int[] Orient(Pos with)
        => new [] {
            S(x, with.x), S(x, with.y), S(x, with.z),
            S(y, with.x), S(y, with.y), S(y, with.z),
            S(z, with.x), S(z, with.y), S(z, with.z)};
}

record Relative(int from, int to, Pos diff) {
    public override string ToString()
        => $"{from}>{to} {diff}";
}

record Scanner(int id, Pos[] beacons) {
    public Pos pos;
    public int[] orient = { 1, 0, 0, 0, 1, 0, 0, 0, 1};

    public Pos Transformed(int b)
        => beacons[b].Reorient(orient).Add(pos);
    public Pos[] Transformed()
        => beacons.Select(b => b.Reorient(orient).Add(pos)).ToArray();

    public Relative[] Distances()
        => beacons.SelectMany((f, i) => beacons
                .Select((t, j) => new Relative(i, j, f.Diff(t)))
                .Where(p => p.to != p.from))
            .GroupBy(p => (p.from, p.to))
            .Select(g => g.First())
            .OrderBy(p => p.diff.Mag())
            .ToArray();

    public override string ToString()
        => $"{id}: {pos} {Show(orient)}";
}

static string Show<T>(IEnumerable<T> list, string sep = ",")
    => string.Join(sep, list);

var p = new Pos(2, 3, 4);
var o = new[]{0,1,0,0,0,-1,-1,0,0};
var p1 = p.Reorient(o);
var o1 = p.Orient(p1);

Console.WriteLine($"{p} | {Show(o)} | {p1} | {Show(o1)}");
p1.Reorient(o1).Match(p)

2/3/4 | 0,1,0,0,0,-1,-1,0,0 | 3/-4/-2 | 0,0,-1,1,0,0,0,-1,0


True

In [ ]:
record Mapping(Relative a, Relative b, decimal dist) {
    public int[] Orient(int[] orient)
        => a.diff.Reorient(orient).Orient(b.diff);
    public override string ToString()
        => $"{a} ~ {b} {dist}";
}

record Match(int from_a, int from_b, Mapping[] mappings) {
    public override string ToString()
        => $"{from_a} ~ {from_b} #{mappings.Length}";
}

class Beacon {
    public int Id { get; }
    public Beacon(int id)
        => Id = id;
    public Dictionary<int,int> scans { get; } = new();

    public override string ToString()
        => $"{Id}:[{string.Join(",", scans.Select(s => $"{s.Key}->{s.Value}"))}]";
}

Match[] Matches(Scanner a, Scanner b)
    => a.Distances().SelectMany(da =>
        b.Distances().Where(db => da.diff.Mag() == db.diff.Mag())
            .Select(db => new Mapping(da, db, da.diff.Mag())
            ))
        .GroupBy(m => (a: m.a.from, b: m.b.from))
        .Select(m => new Match(m.Key.a, m.Key.b, m.ToArray()))
        .Where(m => m.mappings.Length > 1)
        .OrderByDescending(m => m.mappings.Length)
        .ToArray();


## Input

In [ ]:
using System.IO;
// var input = File.ReadAllLines(@"day-19.sample");
var input = File.ReadAllLines(@"day-19.input");

Scanner[] Parse(string[] i) {
    var s = new List<Scanner>();
    var id = -1;
    var b = new List<Pos>();

    foreach (var l in i) {
        if (string.IsNullOrEmpty(l)) {
            s.Add(new Scanner(id, b.ToArray()));
            continue;
        }
        if (l.Substring(0, 3) == "---") {
            b.Clear();
            var parts = l.Split(' ');
            id = int.Parse(parts[2]);
            continue;
        }
        var p = l.Split(',').Select(int.Parse).ToArray();
        b.Add(new Pos(p[0], p[1], p[2]));
    }
    if (b.Count() > 0) {
        s.Add(new Scanner(id, b.ToArray()));
    }

    return s.ToArray();
}

var scanners = Parse(input);

scanners.Length

27

## Part 1

By finding pairs of scanners that both see at least 12 of the same beacons, you can assemble the entire map.

Assemble the full map of beacons. How many beacons are there?

In [ ]:
scanners[0].pos = new Pos(0, 0, 0);
var known = scanners.Where(s => s.pos != null).ToArray();
var unknown = scanners.Where(s => s.pos == null).ToArray();

while (unknown.Length > 0) {
    display($"{known.Length,2} {unknown.Length,2}");
    var matches = known
        .SelectMany(a => unknown
            .Select(b => (a: a, b: b, matches: Matches(a, b)))
            .Where(t => t.matches.Length > 11)
        ).ToArray();
        
    foreach (var m in matches) {
        var first = m.matches.OrderBy(t => t.from_a).First()
            .mappings.OrderBy(t => t.b.from)
            .First();
        var o = m.b.orient = first.Orient(m.a.orient);
        var pos_a = m.a.Transformed(first.a.from);
        var bcn_b = m.b.beacons[first.b.from];
        var scn_from = pos_a.Diff(bcn_b.Reorient(o));
        m.b.pos = scn_from;
        // display($"{m.a}\n- {first.a} {pos_a}\n- {first.b} {bcn_b}\n> {m.b} {scn_from}");
    }
    known = scanners.Where(s => s.pos != null).ToArray();
    unknown = scanners.Where(s => s.pos == null).ToArray();
}

display(scanners.Select(s => $"{s.id}: {s.pos} {Show(s.orient)}"));

scanners.SelectMany(s => s.Transformed()).DistinctBy(p => $"{p}").Count()


 1 26

 2 25

 5 22

10 17

15 12

19  8

24  3

index,value
0,"0: 0/0/0 1,0,0,0,1,0,0,0,1"
1,"1: 1322/-2282/1116 0,1,0,-1,0,0,0,0,1"
2,"2: 2403/-2246/1125 0,-1,0,1,0,0,0,0,1"
3,"3: -1028/-7104/-25 0,-1,0,0,0,-1,1,0,0"
4,"4: 55/-3610/2500 0,-1,0,0,0,1,-1,0,0"
5,"5: 2512/-1193/-1206 0,0,-1,1,0,0,0,-1,0"
6,"6: -1083/-1099/-1143 1,0,0,0,0,-1,0,1,0"
7,"7: 179/-2321/1186 -1,0,0,0,0,-1,0,-1,0"
8,"8: 3669/-2272/1278 0,0,1,0,-1,0,1,0,0"
9,"9: 3625/-3552/1237 -1,0,0,0,-1,0,0,0,1"


353

## Part 2

Sometimes, it's a good idea to appreciate just how big the ocean is. Using the Manhattan distance, how far apart do the scanners get?

What is the largest Manhattan distance between any two scanners?

In [ ]:
var p1 = new Pos(1105,-1205,1229);
var p2 = new Pos(-92,-2380,-20);

var m = p1.Diff(p2);
display(m.x + m.y + m.z);

scanners.SelectMany(a => 
    scanners.Where(b => b.id != a.id)
        .Select(b => (a_id: a.id, b_id: b.id, diff: a.pos.Diff(b.pos)))
    )
    .Select(t => (t, dist: t.diff.x + t.diff.y + t.diff.z))
    .MaxBy(t => t.dist)

3621

Item1,Item2
"( 8, 3, { 4697/4832/1303: x: 4697, y: 4832, z: 1303 } )",10832
